In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Homework1

Mounted at /gdrive
/gdrive/My Drive/Homework1


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import matplotlib.pyplot as plt
from random import randint

In [3]:
seed = 19
import os
import random

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
def load_data(folder="public_data.npz", resolution=96, head_only=False):
    images = []

    loaded = np.load(folder, allow_pickle=True)

    # Iterate through files in the specified folder
    for i, img in enumerate(loaded['data']):
        # Normalize image pixel values to a float range [0, 1]
        #img = (img / 255).astype(np.float32)

        # Convert image from BGR to RGB
        #img = img[...,::-1]

        # Make the image dataset squared
        dim = min(img.shape[:-1])
        img = img[(img.shape[0]-dim)//2:(img.shape[0]+dim)//2, (img.shape[1]-dim)//2:(img.shape[1]+dim)//2, :]

        # Resize the image to 224x224 pixels
        #img = tfkl.Resizing(224, 224)(img)
        img = tfkl.Resizing(resolution, resolution)(img)

        if img is not None:
            images.append(img)

        if (head_only and i == 9):
           break

    labels = loaded['labels']
    loaded.close()

    if (head_only):
       labels = labels[:10]

    y = LabelEncoder().fit_transform(labels)
    y = tfk.utils.to_categorical(y, 2)

    return np.array(images), y




def display_random_images(X, y, num_img=10):
  # Create subplots for displaying items
  fig, axes = plt.subplots(2, num_img//2, figsize=(20, 9))
  for i in range(num_img):
      image = randint(0, X.shape[0] - 1)

      ax = axes[i%2, i%num_img//2]
      ax.imshow(np.clip(X[image], 0, 255))  # Display clipped item images
      ax.text(0.5, -0.1, str(image) + ' ' + str(y[image]), size=12, ha="center", transform=ax.transAxes)
      ax.axis('off')
  plt.tight_layout()
  plt.show()




def delete_outliers(X, y):
  shrek = 137
  trololo = 5143

  new_X = []
  new_y = []

  num_outliers = 0

  for i, sample in enumerate(X):
    if (not (np.array_equal(sample, X[shrek]) or np.array_equal(sample, X[trololo]))):
      new_X.append(sample)
      new_y.append(y[i])
    else:
      num_outliers += 1

  return np.array(new_X), np.array(new_y), num_outliers

In [5]:
X, y = load_data('public_data.npz')
X, y, num_outliers = delete_outliers(X, y)

In [6]:
# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=600, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train_0, y_val = train_test_split(X_train_val, y_train_val, test_size=600, stratify=np.argmax(y_train_val,axis=1))

print(X_train.shape, y_train_0.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(3804, 96, 96, 3) (3804, 2)
(600, 96, 96, 3) (600, 2)
(600, 96, 96, 3) (600, 2)


In [7]:
augment1 = tf.keras.Sequential([
    tfkl.RandomFlip(),
    tfkl.RandomTranslation(height_factor = (-0.2,0.2), width_factor = (-0.2,0.2), fill_mode = 'reflect'),
    tfkl.RandomZoom(0.3, fill_mode = 'reflect'),
    tfkl.RandomBrightness(0.1, value_range=(0,255)),
])

augment2 = tf.keras.Sequential([
    tfkl.RandomFlip(),
    tfkl.RandomZoom(0.4, fill_mode = 'reflect'),
    tfkl.RandomBrightness(0.1, value_range=(0,255)),
    tfkl.RandomRotation((-1,1), fill_mode = 'reflect'),
])

new_X_train_1 = augment2(X_train[np.where((y_train_0[:, 0] == 0) & (y_train_0[:, 1] == 1))])
augmented_X_train_2 = augment2(X_train)
augmented_X_train_1 = augment1(augmented_X_train_2)

X_train = np.append(X_train ,augmented_X_train_2, axis = 0)
X_train = np.append(X_train ,augmented_X_train_1, axis = 0)
X_train = np.append(X_train ,new_X_train_1, axis = 0)

y_train = np.append(y_train_0, y_train_0, axis = 0)
y_train = np.append(y_train, y_train_0, axis = 0)
for k in range(new_X_train_1.shape[0]):
    y_train = np.append(y_train, [[0,1]], axis = 0)

In [8]:
# Define key model parameters
input_shape = X_train.shape[1:]  # Input shape for the model
output_shape = y_train.shape[1]  # Output shape for the model
batch_size = 16                # Batch size for training, always a power of 2!!
epochs = 400

In [9]:
mobile = tfk.applications.EfficientNetV2B0(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet",
    pooling='avg',
    include_preprocessing = True
)
mobile.trainable = False

24274472/24274472 [==============================] - 0s 0us/step


In [10]:
inputs = tfk.Input(shape=input_shape)
# Connect MobileNetV2 to the input
x = mobile(inputs)
x = tfkl.Dropout(rate = 1/5)(x)
x = tfkl.Dense(64)(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(rate = 1/7)(x)
x = tfkl.Dense(16)(x)
x = tfkl.BatchNormalization()(x)
x = tfkl.Activation('relu')(x)
x = tfkl.Dropout(rate = 1/7)(x)
# Add a Dense layer with 2 units and softmax activation as the classifier
outputs = tfkl.Dense(2, activation='softmax')(x)

# Create a Model connecting input and output
model_1 = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with Categorical Cross-Entropy loss and Adam optimizer
model_1.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(learning_rate=1e-3, weight_decay=3e-5), metrics=['accuracy'])

# Display model summary
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 efficientnetv2-b0 (Functio  (None, 1280)              5919312   
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 64)                81984     
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 64)                0     

In [12]:
history = model_1.fit(
    x = X_train, # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 400,
    validation_data = (X_val, y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.9, patience = 8, min_lr = 2e-6)
    ]
).history

Epoch 1/400
804/804 [==============================] - 42s 26ms/step - loss: 0.5882 - accuracy: 0.6882 - val_loss: 0.3897 - val_accuracy: 0.8167 - lr: 0.0010
Epoch 2/400
804/804 [==============================] - 16s 19ms/step - loss: 0.5171 - accuracy: 0.7450 - val_loss: 0.3766 - val_accuracy: 0.8183 - lr: 0.0010
Epoch 3/400
804/804 [==============================] - 13s 16ms/step - loss: 0.4901 - accuracy: 0.7628 - val_loss: 0.3558 - val_accuracy: 0.8300 - lr: 0.0010
Epoch 4/400
804/804 [==============================] - 14s 17ms/step - loss: 0.4800 - accuracy: 0.7709 - val_loss: 0.3480 - val_accuracy: 0.8567 - lr: 0.0010
Epoch 5/400
804/804 [==============================] - 14s 17ms/step - loss: 0.4728 - accuracy: 0.7736 - val_loss: 0.3379 - val_accuracy: 0.8683 - lr: 0.0010
Epoch 6/400
804/804 [==============================] - 14s 17ms/step - loss: 0.4611 - accuracy: 0.7830 - val_loss: 0.3403 - val_accuracy: 0.8500 - lr: 0.0010
Epoch 7/400
804/804 [==============================]

In [13]:
# Evaluate the model on the test set
test_accuracy = model_1.evaluate(X_test,y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)

Test set accuracy 0.8550


In [14]:
model_1.save('efficientnetv2b0_and_augmentation_NO_INVERSION')

In [15]:
del model_1

In [16]:
ft_model = tfk.models.load_model('efficientnetv2b0_and_augmentation_NO_INVERSION')

In [18]:
ft_model.get_layer('efficientnetv2-b0').trainable = True

In [20]:
ft_model.get_layer('efficientnetv2-b0').summary()

Model: "efficientnetv2-b0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 96, 96, 3)]          0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 96, 96, 3)            0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 96, 96, 3)            0         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 stem_conv (Conv2D)          (None, 48, 48, 32)           864       ['normalizatio

In [21]:
# Freeze first N layers
N = 34    # last block
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b0').layers[:-N]):
  layer.trainable=False

In [23]:
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b0').layers):
  print(layer.name, layer.trainable)

input_1 False
rescaling False
normalization False
stem_conv False
stem_bn False
stem_activation False
block1a_project_conv False
block1a_project_bn False
block1a_project_activation False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
block2a_project_conv False
block2a_project_bn False
block2b_expand_conv False
block2b_expand_bn False
block2b_expand_activation False
block2b_project_conv False
block2b_project_bn False
block2b_drop False
block2b_add False
block3a_expand_conv False
block3a_expand_bn False
block3a_expand_activation False
block3a_project_conv False
block3a_project_bn False
block3b_expand_conv False
block3b_expand_bn False
block3b_expand_activation False
block3b_project_conv False
block3b_project_bn False
block3b_drop False
block3b_add False
block4a_expand_conv False
block4a_expand_bn False
block4a_expand_activation False
block4a_dwconv2 False
block4a_bn False
block4a_activation False
block4a_se_squeeze False
block4a_se_reshape False
block4a

In [24]:
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(), metrics='accuracy')

In [25]:
history = ft_model.fit(
    x = X_train, # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 16,
    epochs = 400,
    validation_data = (X_val, y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.9, patience = 8, min_lr = 2e-6)
    ]
).history

Epoch 1/400
804/804 [==============================] - 37s 28ms/step - loss: 0.5023 - accuracy: 0.7606 - val_loss: 0.3145 - val_accuracy: 0.8633 - lr: 0.0010
Epoch 2/400
804/804 [==============================] - 22s 27ms/step - loss: 0.4102 - accuracy: 0.8179 - val_loss: 0.3032 - val_accuracy: 0.8533 - lr: 0.0010
Epoch 3/400
804/804 [==============================] - 20s 25ms/step - loss: 0.3526 - accuracy: 0.8506 - val_loss: 0.2957 - val_accuracy: 0.8683 - lr: 0.0010
Epoch 4/400
804/804 [==============================] - 19s 24ms/step - loss: 0.3238 - accuracy: 0.8632 - val_loss: 0.3143 - val_accuracy: 0.8583 - lr: 0.0010
Epoch 5/400
804/804 [==============================] - 23s 28ms/step - loss: 0.2941 - accuracy: 0.8788 - val_loss: 0.3518 - val_accuracy: 0.8517 - lr: 0.0010
Epoch 6/400
804/804 [==============================] - 21s 26ms/step - loss: 0.2648 - accuracy: 0.8943 - val_loss: 0.3267 - val_accuracy: 0.8650 - lr: 0.0010
Epoch 7/400
804/804 [==============================]

In [26]:
# Evaluate the model on the test set
test_accuracy = ft_model.evaluate(X_test,y_test,verbose=0)[-1]
print('Test set accuracy %.4f' % test_accuracy)

Test set accuracy 0.8783


In [27]:
ft_model.save('Efficientnet_finetuned_NO_INVERSION')